In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, DepthwiseConv2D

Question 1 -
Implement 3 different CNN architectures with a comparison table for the MNSIT
dataset using the Tensorflow library.
Note -
1. The model parameters for each architecture should not be more than 8000
parameters
2. Code comments should be given for proper code understanding.
3. The minimum accuracy for each accuracy should be at least 96%

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
x_train.shape

(60000, 28, 28)

In [ ]:
# Normalise the pixel values to range 0-1 and reshape
x_train = x_train.reshape((x_train.shape[0], 28,28,1)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0],28,28,1)).astype('float32') / 255

In [ ]:
#one hot encoding the lables
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess the dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize and reshape the data
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)).astype('float32') / 255

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Architecture 1: Small CNN with 6,810 parameters
model1 = models.Sequential([
    layers.Conv2D(4, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])

# Architecture 2: Depthwise CNN with 6,650  parameters
model2 = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.DepthwiseConv2D((3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])

# Architecture 3: Strided CNN with 5,930 parameters
model3 = models.Sequential([
    layers.Conv2D(20, (3, 3), strides=(2,2), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])


# Function to compile, train, and evaluate models
def train_and_evaluate(model, x_train, y_train, x_test, y_test, optimizer='adam', epochs=5, batch_size=64):
    print("optimizer", optimizer, "epochs", epochs, "batch_size", batch_size)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    return test_acc, model.count_params()

# Train and evaluate all models
results = {}
models_list = [model1, model2, model3]
model_names = ['Model 1', 'Model 2', 'Model 3']
optimizers = ['adam', 'sgd', 'rmsprop']
epochs = [15, 10, 7]

for model, name, optimizer, epoch in zip(models_list, model_names, optimizers, epochs):
    acc, params = train_and_evaluate(model, x_train, y_train, x_test, y_test, optimizer, epoch)
    results[name] = {'Accuracy': acc, 'Parameters': params, 'Optimizer':optimizer}

# Print results in a comparison table
print(f"{'Model':<10} {'Accuracy':<10} {'Parameters':<10} {'Optimizer':<10}")
for model, metrics in results.items():
    print(f"{model:<10} {metrics['Accuracy']:<10.4f} {metrics['Parameters']:<10} {metrics['Optimizer']:10}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


optimizer adam epochs 15 batch_size 64
optimizer sgd epochs 10 batch_size 64
optimizer rmsprop epochs 7 batch_size 64
Model      Accuracy   Parameters Optimizer 
Model 1    0.9728     6810       adam      
Model 2    0.9758     6650       sgd       
Model 3    0.9732     7410       rmsprop   


Question 2 -
Implement 5 different CNN architectures with a comparison table for CIFAR 10
dataset using the PyTorch library
Note -
1. The model parameters for each architecture should not be more than 10000
parameters
2. Code comments should be given for proper code understanding

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchsummary import summary
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from time import time

In [ ]:
#Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"using Device {device}")
if torch.cuda.is_available():
  print(f"cuda name is {torch.cuda.get_device_name(0)}")
# Hyperparameters
batch_size = 32
learning_rate = 0.001

# Data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

print(train_dataset.data.shape[1:])
print(train_loader.dataset.data.shape[1:])

using Device cpu


100%|██████████| 170M/170M [00:03<00:00, 48.5MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
(32, 32, 3)
(32, 32, 3)


In [ ]:
def count_trainable_params(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
class CNN1(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(in_channels=8, out_channels=13, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(13 * 8 * 8, 10)
    )
    self.to(device)


  def forward(self, x):
    x = self.conv1(x)
    return x


class CNN2(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1), # (input_width - Filter_size + 2(padding)) / Stripe + 1 = (32 - 3 + 2(0)) / 2 + 1
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2), # = (16-2)/1 +1
        nn.Conv2d(in_channels=16, out_channels=31, kernel_size=3, stride=2, padding=1), # (16 - 3)  / 1 +1
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(31 * 4 * 4, 10)
    )
    self.to(device)
  def forward(self, x):
    x = self.conv1(x)
    return x


class CNN3(nn.Module):
  def __init__(self):
    super(CNN3, self).__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=14, kernel_size=3, stride=2, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(14 * 8 * 8, 10)
    )
    self.to(device)

  def forward(self, x):
      x = self.conv1(x)
      return x

class CNN4(nn.Module):
  def __init__(self):
    super(CNN4, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=2, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(16 * 7 * 7, 10)
    )
    self.to(device)

  def forward(self, x):
    x = self.conv(x)
    return x

class CNN5(nn.Module):
  def __init__(self):
    super(CNN5, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(in_channels=6, out_channels=14, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(14 * 8 * 8, 10)
    )

  def forward(self, x):
    x = self.conv(x)
    return x


In [ ]:
def train_and_evaluate(model, optimizer_name='adam', epochs=5, lr=0.001):
  model_name = model.__class__.__name__
  criterion = nn.CrossEntropyLoss()
  if optimizer_name.lower() == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
  elif optimizer_name.lower() == 'sgd':
      optimizer = optim.SGD(model.parameters(), lr=lr)
  elif optimizer_name.lower() == 'rms':
      optimizer = optim.RMSprop(model.parameters(), lr=lr)
  elif optimizer_name.lower() == 'adadelta':
      optimizer = optim.Adadelta(model.parameters(), lr=lr)
  elif optimizer_name.lower() == 'adagrad':
      optimizer = optim.Adagrad(model.parameters(), lr=lr)
  else:
      raise ValueError("Invalid optimizer name. Choose from 'adam', 'sgd', 'rms', 'adadelta' or 'adagrad'.")
  start_time = time()
  for epoch in range(epochs):
      for i, (images, labels) in enumerate(train_loader):
          images = images.to(device)
          labels = labels.to(device)

          # Forward pass
          outputs = model(images)
          loss = criterion(outputs, labels)

          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          if (i + 1) % 1500 == 0:
              print(f'{model_name} Epoch [{epoch + 1}/{epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

  end_time = time()
  print(f"Model name: {model_name}, Training time: {(end_time - start_time):.2f}s")

  # Evaluation
  model.eval()
  with torch.no_grad():
      correct = 0
      total = 0
      for images, labels in test_loader:
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

      accuracy = 100 * correct / total
      print(f'Accuracy of the model {model_name} on the test images: {accuracy:.2f}%')
      return accuracy, count_trainable_params(model), end_time - start_time


In [7]:
# Train and evaluate all models
results = {}
models_list = [CNN1(), CNN2(), CNN3(), CNN4(), CNN5()]
model_names = ['Model_1','Model_2', 'Model_3', 'Model_4', 'Model_5']
optimizers = ['rms', 'adam', 'sgd', 'adadelta', 'adagrad']
epochs = [5, 5, 10, 10, 10]

for model, name, optimizer, epoch in zip(models_list, model_names, optimizers, epochs):
    acc, params, training_time = train_and_evaluate(model, optimizer, epoch)
    results[name] = {'Accuracy': acc, 'Parameters': params, 'Optimizer':optimizer, 'training_time': training_time}

# Print results in a comparison table
print(f"{'Model':<10} {'Accuracy':<10} {'Parameters':<10} {'Optimizer':<10} {'Training time':<10}")
for model, metrics in results.items():
    print(f"{model:<10} {metrics['Accuracy']:<10.3f} {metrics['Parameters']:<10} {metrics['Optimizer']:10} {metrics['training_time']:10.2f}")

CNN1 Epoch [1/5], Step [1500/1563], Loss: 1.4610
CNN1 Epoch [2/5], Step [1500/1563], Loss: 1.0808
CNN1 Epoch [3/5], Step [1500/1563], Loss: 1.0839
CNN1 Epoch [4/5], Step [1500/1563], Loss: 0.6931
CNN1 Epoch [5/5], Step [1500/1563], Loss: 1.1944
Model name: CNN1, Training time: 167.77s
Accuracy of the model CNN1 on the test images: 58.99%
CNN2 Epoch [1/5], Step [1500/1563], Loss: 1.2476
CNN2 Epoch [2/5], Step [1500/1563], Loss: 1.4029
CNN2 Epoch [3/5], Step [1500/1563], Loss: 1.1941
CNN2 Epoch [4/5], Step [1500/1563], Loss: 1.2674
CNN2 Epoch [5/5], Step [1500/1563], Loss: 0.7123
Model name: CNN2, Training time: 203.52s
Accuracy of the model CNN2 on the test images: 64.75%
CNN3 Epoch [1/10], Step [1500/1563], Loss: 1.9556
CNN3 Epoch [2/10], Step [1500/1563], Loss: 2.1128
CNN3 Epoch [3/10], Step [1500/1563], Loss: 1.7514
CNN3 Epoch [4/10], Step [1500/1563], Loss: 1.7471
CNN3 Epoch [5/10], Step [1500/1563], Loss: 1.5929
CNN3 Epoch [6/10], Step [1500/1563], Loss: 1.7503
CNN3 Epoch [7/10], S